# Demonstrate ``TuneAxis``

In this example, we demonstrate the `apstools.plans.TuneAxis()` plan.  The `TuneAxis()` support may be used to align (a.k.a. *tune*) a signal against an axis.

We'll use a software-only (not connected to hardware) motor as a positioner.  Here, we prepare a signal that is a computation based on the value of our positioner.  The computed signal is a model of a realistic diffraction peak ([pseudo-Voigt](https://en.wikipedia.org/wiki/Voigt_profile), a mixture of a Gaussian and a Lorentzian) one might encounter in a powder diffraction scan.  The model peak is a pseudo-voigt function to which some noise has been added.  Random numbers are used to modify the ideal pseudo-voigt function so as to simulate a realistic signal.

For this demo, we do not need the databroker since we do not plan to review any of this data after collection.  We'll display the data during the scan using the *BestEffortCallback()* code.

In [1]:
from ophyd import EpicsMotor
from apstools.synApps_ophyd import swaitRecord, swait_setup_random_number
from apstools.signals import SynPseudoVoigt
from apstools.plans import TuneAxis
from bluesky.callbacks import LiveTable
import numpy as np
from databroker import Broker
from bluesky import RunEngine

RE = RunEngine({})
db = Broker.named("mongodb_config")
RE.subscribe(db.insert)

0

Figure out which workstation we are running.  The *mint-vm* host has a different IOC prefix.

In [2]:
import socket
if socket.gethostname().find("mint-vm") >= 0:
    prefix = "vm7:"
else:
    prefix = "xxx:"

Connect to our motor *before* we create the simulated detector signal.

In [3]:
m1 = EpicsMotor(prefix+"m1", name="m1")
m1.wait_for_connection()

Define a starting position, we'll use this later in the demo.

In [4]:
m1.move(-1.5)
starting_position = m1.position

## Setup the simulated detector signal.  

Randomize the values a bit so that we have something interesting to find with `TuneAxis()`.

In [5]:
spvoigt = SynPseudoVoigt(
    'spvoigt', m1, 'm1', 
    center=-1.5 + 0.4*np.random.uniform(), 
    eta=0.2 + 0.5*np.random.uniform(), 
    sigma=0.001 + 0.05*np.random.uniform(), 
    scale=1e5,
    bkg=0.01*np.random.uniform())

Reveal the actual values.  These are the answers we expect to discover.

In [6]:
print("spvoigt.scale: ", spvoigt.scale)
print("spvoigt.center: ", spvoigt.center)
print("spvoigt.sigma: ", spvoigt.sigma)
print("spvoigt.eta: ", spvoigt.eta)
print("spvoigt.bkg: ", spvoigt.bkg)

spvoigt.scale:  100000.0
spvoigt.center:  -1.3940973681450914
spvoigt.sigma:  0.025534621641250733
spvoigt.eta:  0.2993015167776747
spvoigt.bkg:  0.0045580721055284755


We will add the actual values as metadata to these scans.

In [7]:
md = dict(
    activity = "TuneAxis development and testing",
    peak_model = "pseudo Voigt",
    peak_scale = spvoigt.scale,
    peak_center = spvoigt.center,
    peak_sigma = spvoigt.sigma,
    peak_eta = spvoigt.eta,
    peak_bkg = spvoigt.bkg
    )

## Set up the tuner

Create a *TuneAxis()* object.  The *tuner* needs to know the positioner, what range to scan to find the peak, *and* it needs the name of the signal to be scanned (since the signal list may have more than one signal).

In [8]:
tuner = TuneAxis([spvoigt], m1, signal_name=spvoigt.name)
tuner.width = 2.5
tuner.step_factor = tuner.num/2.5

Configure the *LiveTable* to also show the simulated detector signal.

In [9]:
live_table = LiveTable(["m1", "spvoigt"])
#spvoigt.read_attrs = ["value"]

# Multi-pass tune

Execute multiple passes to refine the centroid determination.
Each subsequent pass will reduce the width of the next scan by ``step_factor``.

In [10]:
RE(tuner.multi_pass_tune(), live_table, md=md)

+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |    spvoigt |
+-----------+------------+------------+------------+
|         1 | 17:12:09.7 |   -2.75000 |    466.418 |
|         2 | 17:12:10.2 |   -2.47000 |    472.656 |
|         3 | 17:12:10.7 |   -2.19000 |    486.582 |
|         4 | 17:12:11.2 |   -1.92000 |    526.201 |
|         5 | 17:12:11.7 |   -1.64000 |    775.096 |
|         6 | 17:12:12.2 |   -1.36000 |  39939.541 |
|         7 | 17:12:12.7 |   -1.08000 |    652.314 |
|         8 | 17:12:13.2 |   -0.81000 |    512.898 |
|         9 | 17:12:13.7 |   -0.53000 |    481.921 |
|        10 | 17:12:14.2 |   -0.25000 |    470.709 |
+-----------+------------+------------+------------+
generator TuneAxis.multi_pass_tune ['7f7884bf'] (scan num: 1)
x : m1
y : spvoigt
cen : -1.3602204901822772
com : -1.3756508139688854
fwhm : 0.28176586200160525
min : [  -2.75        466.41822165]
max : [ -1.36000000e+00   3.99395408e+04]
crossings : [-1.5011

('7f7884bf-c59f-49d4-aa25-6e0c5019b813',
 'f0658421-7b7e-45ef-a724-0b10538ec98d',
 '37c188c0-4f24-4e9b-b8ab-d610dc8797c5',
 '44f573de-35d4-4dc2-98fd-a44bd585e30b')

Show the results from the multi-pass tuning.

In [11]:
print("final: ", tuner.center)
print("max", tuner.peaks.max)
print("min", tuner.peaks.min)
for stat in tuner.stats:
    print("--", stat.cen, stat.fwhm)
print("m1=", m1.position, "", "det=", spvoigt.get())

final:  -1.39403315527
max (-1.3900000000000001, 98808.184210417196)
min (-1.3700000000000001, 61175.978285094367)
-- -1.36022049018 0.281765862002
-- -1.39254935644 0.0696594765635
-- -1.39403315527 0.0586657457215
-- -1.39413020313 0.0314478237932
m1= -1.3900000000000001  det= 79738.9517242


Compare the final position (just printed) with the expected value shown a couple steps back.

## Single-pass tune

Repeat but with only one pass.  Reset the motor to the starting position and increase the number of steps by a factor of three.

In [12]:
m1.move(starting_position)
tuner.num *= 3
RE(tuner.tune(), live_table, md=md)

+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |    spvoigt |
+-----------+------------+------------+------------+
|         1 | 17:12:25.0 |   -2.75000 |    466.418 |
|         2 | 17:12:25.2 |   -2.66000 |    467.980 |
|         3 | 17:12:25.5 |   -2.58000 |    469.677 |
|         4 | 17:12:25.8 |   -2.49000 |    472.047 |
|         5 | 17:12:26.1 |   -2.41000 |    474.704 |
|         6 | 17:12:26.4 |   -2.32000 |    478.553 |
|         7 | 17:12:26.7 |   -2.23000 |    483.710 |
|         8 | 17:12:27.0 |   -2.15000 |    489.922 |
|         9 | 17:12:27.3 |   -2.06000 |    499.752 |
|        10 | 17:12:27.6 |   -1.97000 |    514.531 |
|        11 | 17:12:27.9 |   -1.89000 |    534.952 |
|        12 | 17:12:28.2 |   -1.80000 |    573.787 |
|        13 | 17:12:28.5 |   -1.72000 |    638.421 |
|        14 | 17:12:28.8 |   -1.63000 |    802.419 |
|        15 | 17:12:29.1 |   -1.54000 |   1345.299 |
|        16 | 17:12:29.4 |   -1.46000 |   6869

('957d83c1-9f11-47df-936b-ce14a396cd76',)

Compare the single-pass scan with the previous multi-pass scan.  Each used the same number of points overall.  

The results are comparable but we already knew the position of the peak approximately.

In [13]:
print("final: ", tuner.center)
print("max", tuner.peaks.max)
print("min", tuner.peaks.min)
print("centroid", tuner.peaks.cen)
print("FWHM", tuner.peaks.fwhm)
print("m1=", m1.position, "", "det=", spvoigt.value)

final:  -1.37211380384
max (-1.3700000000000001, 61175.978285094367)
min (-2.75, 466.41822164640951)
centroid -1.37211380384
FWHM 0.0963834056615
m1= -1.37  det= 470.708557361
